In [3]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-ollama
%pip install llama-index-vector-stores-chroma

In [5]:
#!pip install chromadb
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Import the necessary module
from dotenv import load_dotenv
import os

# Load environment variables from the .env file (if present)
load_dotenv(override=True)
docs_path = os.getenv('DOCS_PATH')
ollama_host = os.getenv('OLLAMA_HOST_URL')
vectordb_path = os.getenv('VECTOR_DB')
print(f"""
    Environment variable are
    docs_path = {docs_path}
    ollama_host = {ollama_host}
    vectordb_path = {vectordb_path}
""")

In [ ]:

from IPython.display import Markdown, display
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [16]:
# create client and a new collection
collections_name="rbi-master-directions-new"
chroma_client = chromadb.PersistentClient(path=vectordb_path)
#chroma_client.delete_collection(name=collections_name)
chroma_collection = chroma_client.get_or_create_collection(name=collections_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [17]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")
local_model_dir = "./bge_onnx"
embed_model = OptimumEmbedding(folder_name=local_model_dir)

In [18]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader(docs_path).load_data()

# bge embedding model
Settings.embed_model = embed_model

llm= Ollama(model="llama3", request_timeout=300.0, base_url=ollama_host)

# ollama
Settings.llm = llm

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [12]:
query_engine = index.as_query_engine(similarity_top_k=4)
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [13]:
response = query_engine.query("What is the provision percentage of sub standard assets?")
display(Markdown(str(response)))

Based on the context information provided, I'll try to answer your question step by step.

From the text, we can see that there are two types of substandard assets mentioned: one for general and another for unsecured exposures. Let's look at each case:

**General Substandard Assets (5.4 i)**

According to paragraph 5.4(i), a general provision of **10 percent** on total outstanding should be made without making any allowance for ECGC guarantee cover and securities available.

**Unsecured Exposures Classified as 'Substandard' (5.4 ii)**

For unsecured exposures classified as 'substandard', an additional provision of **10 per cent** is required, which makes a total of **20 percent** on the outstanding balance.

Now, let's summarize: for general substandard assets, the provision percentage is 10%, and for unsecured exposures classified as 'substandard', it's 20%.

So, to answer your question:

The provision percentage of substandard assets is **10% (general)** or **20% (unsecured exposures)**.

In [10]:
response = query_engine.query("what are the asset classification guidelines related to project finance?")
print(response)

Based on the provided context information, the asset classification guidelines related to project finance are as follows:

4.2.1 Broadly speaking, classification of assets into above categories should be done taking into account the degree of well-defined credit weaknesses and the extent of dependence on collateral security for realisation of dues.

4.2.2 Banks should establish appropriate internal systems to eliminate the tendency to delay or postpone the identification of NPAs, especially in respect of high value accounts. The banks may fix a minimum cut off point to decide what would constitute a high value account depending upon their respective business levels.

4.2.3 Availability of security / net worth of borrower/ guarantor should not be taken into account for the purpose of treating an advance as NPA or otherwise, except to the extent provided in Para 4.2.9, as income recognition is based on record of recovery.

4.2.9 Accounts with temporary deficiencies: The classification of

In [11]:
response = query_engine.query("What is the provision percentage of doubtful assets?")
print(response)

Based on the provided context information, the provision percentage for doubtful assets is 100%.


In [12]:
response = query_engine.query("is there any provision for standard assets?")
print(response)

Yes, there are provisions for standard assets in DBOD-MC on IRAC Norms-2010. According to the document, the provisioning requirements for all types of standard assets stands amended as follows, with effect from November 5, 2009:

* Direct advances to agricultural and SME sectors at 0.25%
* Advances to Commercial Real Estate (CRE) sector at 1.00%
* All other loans and advances not included in (a) and (b) above at 0.40%

Additionally, the document states that while the provisions on individual portfolios are required to be calculated at the rates applicable to them, the excess or shortfall in the provisioning, vis-à-vis the position as on any previous date, should be determined on an aggregate basis.


In [14]:
response = query_engine.query("when can an account be classified as substandard?")
print(response)

An account can be classified as substandard when it has remained nonperforming for a period of less than or equal to 12 months from the date of disbursement, according to DBOD-MC on IRAC Norms-2010. This is based on the assumption that the borrower's credit weaknesses are well defined and jeopardize the liquidation of the debt, but there is still a possibility of recovery with adequate provisions being made. The account will be classified as substandard after a period of 12 months from the date of disbursement, as per the IRAC Norms-2010.


In [15]:
response = query_engine.query("What is the provision percentage incase of doubtful assets if there is secured and unsecured portion ?")
print(response)

Based on the given context information, the provision percentage for doubtful assets with both secured and unsecured portions would be 100% for the entire asset. This is because the guidelines provided in DBOD-MC on IRAC Norms-2010 require banks to provide for 100% of the net investment in the lease and the unrealized portion of finance income net of finance charge component for substandard assets, including those with both secured and unsecured portions.

Therefore, the provision percentage for doubtful assets would be 100% of the total outstanding balance, which includes both the secured and unsecured portions.
